MOTIFENUMERATION(Dna, k, d)
    Patterns ← an empty set
    for each k-mer Pattern in Dna
        for each k-mer Pattern’ differing from Pattern by at most d mismatches
            if Pattern' appears in each string from Dna with at most d mismatches
                add Pattern' to Patterns
    remove duplicates from Patterns
    return Patterns

[Brute Force algorithm]


In [7]:
def mutate(pattern, d):
    def helper(input, n, remaining_d):                     # helper함수가 재귀식이 가능토록 함
        if remaining_d == 0:                               # d번 변형을 순차적으로 할 것이므로, d번 이상/이하로 변형 안되도록
            if input != pattern:                           # 변형된 값이 기존 pattern이랑 달라야 함
                mutants.add(input)                         # d번 변형 완료 시, mutants에 input저장 (modify가 아닌 이유는 마지막에 modify를 input으로 받음)
            return
        
        if n == len(pattern):                              # pattern의 길이와 변형할 위치가 같아지면 (예: 3자리 유전자 n=3번째 자리 도착) 끝내기
            return                                         # 이 두번째 if문은 위의 첫번째 if문이 True여야 진행함
        
        for char in "ACGT":                                # A,C,G,T 중 1개로 변형할 것임
            modify = input[:n] + char + input[n+1:]        # modify에 n번째 이전까지 문자열 + 변형n번째 + n번째 이후 문자열로 변형된 문자 저장
            helper(modify, n+1, remaining_d-1)             # n+1은 똑같은 자리X 다른 자리를 바꾸기 위해, remain_d는 얼마나 변형 남았는지 횟수, 위로 다시 올라가 for반복
        helper(input, n+1, remaining_d)                    # remain_d가 0일때, 즉 XXOO, XOXO, XOOX (X=바뀐자리)가 완료되면 OXXO와 OXOX를 위해 위로 올려보내기
        
    mutants = set()                                        # 이제 modify의 결과가 이동 된 "input"을 저장할 공간 만들어서
    helper(pattern, 0, d)                                  # 초기에 n="0" 즉 첫번째 문자열 위치부터 시작하기

    # print(f"{pattern}: {sorted(set(mutants))}")           # 실제로 print해보면 k-mer 패턴 별로 d개씩 치환된 set={ , , ...,}을 확인 가능함
    return mutants  

In [8]:
def mutate(pattern, d):
    def helper(input, n, remaining_d):
        if remaining_d == 0:
            if input != pattern:
                mutants.add(input)
            return
        
        if n == len(pattern):
            return
        
        for char in "ACGT":
            modify = input[:n] + char + input[n+1:]
            helper(modify, n+1, remaining_d-1)
        helper(input, n+1, remaining_d)
    
    mutants = set()
    helper(pattern, 0, d)

    return mutants

In [9]:
def Motif_Enumeration(Dna, k, d):
    
    false_pattern_dict = {}                                 # string마다 구분해서 mutants를 가져올거라서 딕셔너리로 
    
    for string in Dna:
        string_results = []
        for i in range(len(string) - k + 1):
            pattern = string[i:i + k]
            false_pattern = mutate(pattern, d)              # 여기까지는 이전처럼 무난하게 k-mer 만드는 것, 이 결과를 위의 mutate함수에 넣으면 mutants를 얻음 
            
            for mutants in false_pattern:                   # 쉽게 구분하려고 여기서도 mutants로 명명
                string_results.append(mutants)              # 각 string을 key값으로 하고, mutants들을 나열한 것을
        
        false_pattern_dict[string] = sorted(set(string_results)) # 알파벳 순으로 sorting해서 처음 만든 딕셔너리에 저장

    # print(sorted(set(false_pattern_dict)))
        
    common = set(false_pattern_dict[Dna[0]])                # 딕셔너리의 첫번째 string을 common으로 가져옴
    # print(common)
    
    
    for i in range(1, len(Dna)):                           # for문을 활용해 Dna해당하는 string 개수만큼 "1"부터 = 첫번째 제외하고 가져옴
        others= Dna[i]
        common = common.intersection(false_pattern_dict[others])
                                                           # 첫번째 string의 mutants들과 나머지 i개의 string에서 교집합을 계속 구함
    return sorted(common)                                  # 즉, 모든 string에서 공통적으로 나타나는 mutant를 알파벳 순서로 정렬

In [10]:
def hamming_distance(mutant, kmer):
    count = 0
    for i in range(len(mutant)):
        if mutant[i] != kmer[i]:
            count += 1
    return count

In [11]:
def motif_enumeration(Dna, k, d):
    patterns = set()

    for string in Dna:
        for i in range(len(string) - k + 1):
            pattern = string[i:i + k]
            pattern_mutants = mutate(pattern, d)

            for mutant in pattern_mutants:
                count = 0
                for sequence in Dna:
                    for j in range(len(sequence) - k + 1):
                        kmer = sequence[j:j + k]
                        if hamming_distance(mutant, kmer) <= d:
                            count += 1
                            break
                if count == len(Dna):
                    patterns.add(mutant)

    return list(patterns)

In [12]:
with open("rosalind_ba2a.txt", "r") as file:
    lines = file.readlines()

k, d = map(int, lines[0].split())
Dna = [line.strip() for line in lines[1:]]

for line in lines:
    print(line.strip())

5 1
GAATCTAGAGAGGTTTAGAGCTGAA
TGGGTCCACGGTCCTAATGAAGGTT
CTGTGGGGTTGAGGTACGCTCCGCC
TTTCGTCTTATGATCCGGTTCGGCA
GACAAACTAGAGGTTATGACGCCGT
GAGTTTGGGAACATCAGCTTGGGTT
GGGTCATCGGGAGATGGGTTTGCTC
CAGAAGTAATGTTTTGGGTTTGCTG
CCTTCAGGCCTATGGCGGTTTCGGG
TGGAGACCAGGATATCTACCTGGTT


In [13]:
result = motif_enumeration(Dna, k, d)
print(" ".join(result))

TGGTT GGGTT CGGTT AGGTT
